In [1]:
# Follow example where M3 data is read in. Use example to work with the energy data

import time
import pandas as pd
import numpy as np
from dataclasses import dataclass
from datasetsforecast.m3 import M3
from datasetsforecast.long_horizon import ECL, ETTm1, LongHorizon
import os
os.chdir('C:/Users/WulfN/Python Projects/time_series_model_comparison')
os.environ['NIXTLA_ID_AS_COL'] = '1'

%config interactive_shell.ast_node_interactivity='all'

In [2]:
y_m3, *_= M3.load(directory = '/datasets', group = 'Monthly') # ... Where is the rest of the data, why would you group on unique id
y_m3

,unique_id,ds,y
0,M1,1990-01-31,2640.0
1,M1,1990-02-28,2640.0
2,M1,1990-03-31,2160.0
3,M1,1990-04-30,4200.0
4,M1,1990-05-31,3360.0
...,...,...,...
167557,M999,1993-10-31,5225.9
167558,M999,1993-11-30,5236.3
167559,M999,1993-12-31,5186.6
167560,M999,1994-01-31,5143.4


In [5]:
# Testing with univariate
horizon = 18

y1 = y_m3.groupby('unique_id').tail(horizon) 
x1 = y_m3.drop(y1.index).reset_index(drop=True)

y2 = y_m3.groupby('unique_id').tail(horizon) 
x2 = y_m3.drop(y2.index)

y2 = y2.reset_index(level=0) # index col needed for TimeMixer
x2 = x2.reset_index(level=0) # maybe reset to make it sequentual per unique id? 
# example drops index for x, consider trying that

# OR NIXTLA_ID_AS_COL in environment variable

ECL_dt[1]['ds']

In [148]:
from neuralforecast.core import NeuralForecast
from neuralforecast.losses.pytorch import MAE, MSE
from neuralforecast.models import TimeMixer

simple_tm_model = TimeMixer(input_size=2*horizon, 
                        h=horizon, 
                        n_series=1, 
                        scaler_type='identity', 
                        early_stop_patience_steps=3)

nf = NeuralForecast(models=[simple_tm_model], freq='M')


# results, then loop through each of the data sets
start = time.time()

nf.fit(x1, val_size=horizon) # perhaps alg also validates
preds = nf.predict()

end = time.time()
elapsed_time = round(end - start,0)

# this model is really slow on this simple dataset!
# number of params is very large
# Info as model trains is unusual
    # x2 = 332 seconds (also, no gpu computer)
    # x2 = stops at max_steps = 1000
    # x1 = 283
    # x1 = stops at max_steps = 1000


INFO:lightning_fabric.utilities.seed:Seed set to 1
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
   | Name              | Type                 | Params | Mode 
--------------------------------------------------------------------
0  | loss              | MAE                  | 0      | train
1  | padder            | ConstantPad1d        | 0      | train
2  | scaler            | TemporalNorm         | 0      | train
3  | pdm_blocks        | ModuleList           | 29.2 K | train
4  | preprocess        | SeriesDecomp         | 0      | train
5  | enc_embedding     | DataEmbedding_wo_pos | 2.5 K  | train
6  | normalize_layers 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_steps=1000` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=1000` reached.
INFO: Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:lightning.pytorch.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

In [149]:
elapsed_time


283.0

In [74]:
from neuralforecast.tsdataset import TimeSeriesDataset
from pytorch_forecasting import TimeSeriesDataSet

# Load ECL data
ECL_dt = LongHorizon.load( # get ECL data
    directory="/datasets", group="ECL")

# TimeSeriesDataset (temporal, temporal_cols, indptr, max_size:int,
#                     min_size:int, y_idx:int, static=None,
#                     static_cols=None, sorted=False)

# Maybe combine ECL_dt? 

# dt_x = ECL_dt[1]
# dt_x['ds'] = pd.to_datetime(dt_x['ds'])
# dt_x = dt_x.set_index('ds')

index_vals = list(ECL_dt[1].index) # vector of index vals

# ts_ECL_dt = TimeSeriesDataset(ECL_dt, temporal=, temporal_cols=)
ts_ECL_dt_torch = TimeSeriesDataSet(ECL_dt[1], # dt_x sets date as index, which may not be what's needed)
                                    time_idx = ECL_dt[1]['ds'],
                                    # dt_x.index, 
                                    target = ECL_dt[0]['y'], 
                                    group_ids = ['unique_id', 'ds']# list of all columns OR an id and time column? 
                                    )


KeyError: "None of [Index(['2016-07-01 02:00:00', '2016-07-01 03:00:00', '2016-07-01 04:00:00',\n       '2016-07-01 05:00:00', '2016-07-01 06:00:00', '2016-07-01 07:00:00',\n       '2016-07-01 08:00:00', '2016-07-01 09:00:00', '2016-07-01 10:00:00',\n       '2016-07-01 11:00:00',\n       ...\n       '2019-07-01 16:00:00', '2019-07-01 17:00:00', '2019-07-01 18:00:00',\n       '2019-07-01 19:00:00', '2019-07-01 20:00:00', '2019-07-01 21:00:00',\n       '2019-07-01 22:00:00', '2019-07-01 23:00:00', '2019-07-02 00:00:00',\n       '2019-07-02 01:00:00'],\n      dtype='object', length=8443584)] are in the [columns]"

In [73]:
ECL_dt[1]['ds']

0          2016-07-01 02:00:00
1          2016-07-01 03:00:00
2          2016-07-01 04:00:00
3          2016-07-01 05:00:00
4          2016-07-01 06:00:00
                  ...         
8443579    2019-07-01 21:00:00
8443580    2019-07-01 22:00:00
8443581    2019-07-01 23:00:00
8443582    2019-07-02 00:00:00
8443583    2019-07-02 01:00:00
Name: ds, Length: 8443584, dtype: object

In [25]:
y = ECL_dt[0]
x = ECL_dt[1]

horizon_2 = 24 * 30

test_df = y.groupby('unique_id').tail(horizon_2) # 4 month horizon 24 * 30 * 4 = 2880
train_df = x.drop(test_df.index).reset_index(drop=True) # these are only the training y values


In [30]:
# I don't think marco's example uses multivariate data.
## How does the df need to be labeled to work with the neuralforecast model

# train_df['ds'] = pd.to_datetime(train_df['ds'])
# train_df = train_df.set_index('ds')

test_df # 231120  rows
train_df

,unique_id,ds,ex_1,ex_2,ex_3,ex_4
0,0,2016-07-01 02:00:00,-0.413043,0.166667,-0.500000,-0.001370
1,0,2016-07-01 03:00:00,-0.369565,0.166667,-0.500000,-0.001370
2,0,2016-07-01 04:00:00,-0.326087,0.166667,-0.500000,-0.001370
3,0,2016-07-01 05:00:00,-0.282609,0.166667,-0.500000,-0.001370
4,0,2016-07-01 06:00:00,-0.239130,0.166667,-0.500000,-0.001370
...,...,...,...,...,...,...
8212459,OT,2019-06-01 21:00:00,0.413043,0.333333,-0.500000,-0.086301
8212460,OT,2019-06-01 22:00:00,0.456522,0.333333,-0.500000,-0.086301
8212461,OT,2019-06-01 23:00:00,0.500000,0.333333,-0.500000,-0.086301
8212462,OT,2019-06-02 00:00:00,-0.500000,0.500000,-0.466667,-0.083562


In [27]:
from neuralforecast.core import NeuralForecast
from neuralforecast.losses.pytorch import MAE
from neuralforecast.models import TimeMixer

n_series = train_df.shape[1]

ts_model = TimeMixer(
    input_size=horizon_2 * n_series, 
        # If window size, then try values like 144 (day), 1008 (week), 4320 (month)
    h=horizon_2, # horizon, 15788 of 10 minute steps
    n_series=n_series,
    d_model=16, #dimension of the model, default 32
    # d_ff= #dimension of fully connected network, default 32
    down_sampling_layers=3, # param search
    down_sampling_window=2, # param search
    learning_rate=0.01, # param search
    scaler_type='robust',
    # dropout rate
    early_stop_patience_steps=3,
    max_steps=100,
    loss=MAE(),
    #valid_loss=MAE(),
    batch_size=n_series
    )

INFO:lightning_fabric.utilities.seed:Seed set to 1


In [28]:
nf = NeuralForecast(models=[ts_model], freq='M')
nf.fit(df=train_df, val_size=horizon_2 #, y = ?, targets? 
       ) # need neuralforecast timeseriesDataset object
preds = nf.predict()
preds

KeyError: 'y'